###USE IT TO GENERATE PLOTS AND TABLES###

In [4]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

In [5]:
def preprocess_label(label):  
    # Add spaces before capital letters (if not already present)
    label = re.sub(r'(?<!\s)(?=[A-Z])', ' ', label)
    
    # Replace long names with abbreviations
    label = label.replace("Takahashi Matsuyama", "TM ")
    label = label.replace("T M", "TM ")
    label = label.replace("Dreyfus Wagner", "DW  ")
    label = label.replace("D W", "DW  ")
    label = label.replace("Kou Markowsky Berman", "KMB  ")
    label = label.replace("K M B", "KMB  ")

    # Replace ...
    label = label.replace("Cost_ratio", "Approx-ratio ")
    label = label.replace("worst case", "Worst Case")
    label = label.replace("_", " ")

    # Remove "Time" from the label
    label = label.replace("Time", "")
    label = label.replace("duration", "")
    label = label.replace("TimeToPerformance", "")

    # Remove extra spaces (if any)
    label = re.sub(r'\s+', ' ', label).strip()
    
    return label

In [6]:
def get_subdirectory_from_path(path):
    # Extract the last part of the path (i.e., the folder name)
    return os.path.basename(os.path.normpath(path))   

In [7]:
def split_after_second_space(input_string):
    # Step 1: Insert a space before each capital letter
    modified_string = re.sub(r'([a-z])([A-Z])', r'\1 \2', input_string)
    
    # Step 2: Split the string by spaces
    split_parts = modified_string.split()
    
    # Step 3: Return the string after the second space (i.e., index 2 onward)
    result = " ".join(split_parts[2:])
    return result[0].lower() + result[1:]

In [8]:
# Format numeric values in the columns to remove unnecessary decimals
def format_value(x):
    if isinstance(x, (int, float)):  # Check if the value is numeric
        return f"{x:.0f}" if x == int(x) else f"{x:.3f}"  # Format integers and floats differently
    return str(x)  # Return non-numeric values as strings

In [9]:
def create_approximation_ratio_table(df, x_col, y_cols, opt_col, output_folder="./tables"):
    """
    Generate a table for approximation ratios relative to `opt_col`.

    Parameters:
        df (pd.DataFrame): The input data.
        x_col (str): The independent variable.
        y_cols (list[str]): List of dependent variables for which to calculate approximation ratios.
        opt_col (str): The column representing the "Opt" value.
        output_folder (str): Path to save the LaTeX table.

    Returns:
        pd.DataFrame: Aggregated per-x_col approximation ratios.
        pd.DataFrame: Overall approximation ratio summary.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Calculate approximation ratios
    approx_ratios = df.copy()
    for col in y_cols:
        if col != "worst_case":
            approx_ratios[f"{col}_ratio"] = approx_ratios[col] / approx_ratios[opt_col]
    
    # Per-x_col statistics
    agg_funcs = {f"{col}_ratio": ["mean", "min", "max"] for col in y_cols if col != "worst_case"}
    agg_funcs["worst_case"] = ["mean", "min", "max"]
    per_x_stats = approx_ratios.groupby(x_col).agg(agg_funcs)
    per_x_stats.columns = ["_".join(col).strip() for col in per_x_stats.columns]
    per_x_stats = per_x_stats.reset_index()
    
    # Apply preprocessing to column names in per_x_stats
    per_x_stats.columns = [preprocess_label(col) for col in per_x_stats.columns]
                           
    # Overall statistics
    overall_stats = {}
    for col in y_cols:
        if col != "worst_case":
            ratio_col = f"{col}_ratio"
            overall_stats[ratio_col] = {
                "mean": approx_ratios[ratio_col].mean(),
                "min": approx_ratios[ratio_col].min(),
                "max": approx_ratios[ratio_col].max(),
            }
    # Include overall statistics for worst-case ratio
    overall_stats["worst_case"] = {
        "mean": approx_ratios["worst_case"].mean(),
        "min": approx_ratios["worst_case"].min(),
        "max": approx_ratios["worst_case"].max(),
    }
    overall_stats_df = pd.DataFrame(overall_stats).T.reset_index()
    overall_stats_df.columns = ["Metric", "Mean", "Min", "Max"]

    # Apply preprocessing to the "Metric" column
    overall_stats_df["Metric"] = overall_stats_df["Metric"].apply(preprocess_label)


    # print(overall_stats_df)
    # Apply preprocessing to column names in overall_stats_df
    overall_stats_df.columns = [preprocess_label(col) for col in overall_stats_df.columns]

    # Transpose the table
    per_x_stats_transposed = per_x_stats.T
    per_x_stats_transposed.reset_index(inplace=True)  # Reset the index so the first row can be handled properly
    per_x_stats_transposed.columns = per_x_stats_transposed.iloc[0]  # Set the first row as column headers
    per_x_stats_transposed = per_x_stats_transposed.drop(index=0)  # Remove the old header row
        
    # Use the original x_col name as the table header for clarity
    header_name = preprocess_label(x_col)  # Ensure it's preprocessed consistently
    per_x_stats_transposed.rename(columns={per_x_stats_transposed.columns[0]: header_name}, inplace=True)
    
    # Apply the format_value function to each element of the DataFrame
    for col in per_x_stats_transposed.columns:
        per_x_stats_transposed[col] = per_x_stats_transposed[col].map(format_value)
    
    # Save the transposed table
    per_x_table_path = os.path.join(output_folder, f"approx_ratios_per_x_{x_col}_transposed.tex")
    with open(per_x_table_path, "w") as f:
        f.write(per_x_stats_transposed.to_latex(index=True, float_format="%.3f"))
    
    # Save per-x_col approximation ratios to a LaTeX table
    per_x_table_path = os.path.join(output_folder, f"approx_ratios_per_x_{x_col}.tex")
    with open(per_x_table_path, "w") as f:
        f.write(per_x_stats.to_latex(index=False, float_format="%.3f"))

    # Save overall approximation ratios to a LaTeX table
    overall_table_path = os.path.join(output_folder, f"approx_ratios_overall_{x_col}.tex")
    with open(overall_table_path, "w") as f:
        f.write(overall_stats_df.to_latex(index=False, float_format="%.3f"))

    print(f"Approximation ratio tables saved: {per_x_table_path}, {overall_table_path}")
    return per_x_stats, overall_stats_df

In [10]:
def create_summary_tables(df, x_col, y_cols, opt_col=None, output_folder="./tables"):
    """
    Generate summary statistics tables for y_cols and opt_col, including standard deviation of differences.

    Parameters:
        df (pd.DataFrame): The input data.
        x_col (str): The independent variable.
        y_cols (list[str]): List of dependent variables to process.
        opt_col (str, optional): The column for "Opt" values, if available.
        output_folder (str): Path to save the generated LaTeX tables.
    """
    import os
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Compute per-x_col statistics
    agg_funcs = {col: ["mean", "min", "max"] for col in y_cols}

    if opt_col:
        agg_funcs[opt_col] = ["mean", "min", "max"]
    
    per_x_stats = df.groupby(x_col).agg(agg_funcs)
    per_x_stats.columns = ["_".join(col).strip() for col in per_x_stats.columns]
    per_x_stats = per_x_stats.reset_index()
    
    # Apply preprocessing to column names in per_x_stats
    per_x_stats.columns = [preprocess_label(col) for col in per_x_stats.columns]

    # Compute overall statistics across all x_col values
    overall_stats = {}
    for col in y_cols + ([opt_col] if opt_col else []):
        overall_stats[col] = {
            "mean": df[col].mean(),
            "min": df[col].min(),
            "max": df[col].max(),
        }

    overall_stats_df = pd.DataFrame(overall_stats).T.reset_index()
    overall_stats_df.columns = ["Metric", "Mean", "Min", "Max"]

    # Apply preprocessing to the "Metric" column
    overall_stats_df["Metric"] = overall_stats_df["Metric"].apply(preprocess_label)

    # Apply preprocessing to column names in overall_stats_df
    overall_stats_df.columns = [preprocess_label(col) for col in overall_stats_df.columns]

    # Transpose the table
    per_x_stats_transposed = per_x_stats.T
    per_x_stats_transposed.reset_index(inplace=True)  # Reset the index so the first row can be handled properly
    per_x_stats_transposed.columns = per_x_stats_transposed.iloc[0]  # Set the first row as column headers
    per_x_stats_transposed = per_x_stats_transposed.drop(index=0)  # Remove the old header row
        
    # Use the original x_col name as the table header for clarity
    header_name = preprocess_label(x_col)  # Ensure it's preprocessed consistently
    per_x_stats_transposed.rename(columns={per_x_stats_transposed.columns[0]: header_name}, inplace=True)
    
    # Apply the format_value function to each element of the DataFrame
    for col in per_x_stats_transposed.columns:
        per_x_stats_transposed[col] = per_x_stats_transposed[col].map(format_value)
    
    # Save the transposed table
    per_x_table_path = os.path.join(output_folder, f"per_x_stats_{x_col}_transposed.tex")
    with open(per_x_table_path, "w") as f:
        f.write(per_x_stats_transposed.to_latex(index=True, float_format="%.3f"))
    
    # Save per-x_col statistics to a LaTeX table
    per_x_table_path = os.path.join(output_folder, f"per_x_stats_{x_col}.tex")
    with open(per_x_table_path, "w") as f:
        f.write(per_x_stats.to_latex(index=False, float_format="%.3f"))

    # Save overall statistics to a LaTeX table
    overall_table_path = os.path.join(output_folder, f"overall_stats_{x_col}.tex")
    with open(overall_table_path, "w") as f:
        f.write(overall_stats_df.to_latex(index=False, float_format="%.3f"))

    print(f"Summary tables saved: {per_x_table_path}, {overall_table_path}")
    return per_x_stats, overall_stats_df


In [11]:
# Plotting
def plot_data(df, x_col, y_cols, title, x_label, y_label, output_file=None, opt_col=None, log_scale=False):
    # Aggregate data by x_col (compute mean, min, max for each x)
    agg_funcs = {col: ["mean", "min", "max"] for col in y_cols}
    if opt_col:
        agg_funcs[opt_col] = ["mean", "min", "max"]

    # Group by x_col and compute aggregated values
    aggregated_df = df.groupby(x_col).agg(agg_funcs)
    aggregated_df.columns = ["_".join(col).strip() for col in aggregated_df.columns]
    aggregated_df = aggregated_df.reset_index()

    # Compute the worst-case values
    if opt_col and "worst_case" in y_cols:
        aggregated_df["worst_case_mean"] *= aggregated_df[f"{opt_col}_mean"]
        aggregated_df["worst_case_min"] *= aggregated_df[f"{opt_col}_min"]
        aggregated_df["worst_case_max"] *= aggregated_df[f"{opt_col}_max"]
            

    plt.figure(figsize=(12, 6))

    # Generate up to 10 distinct colors using a colormap
    colormap = plt.get_cmap("tab10", 10)  # Use "tab10" for distinct qualitative colors
    color_list = [colormap(i) for i in range(10)]  # Generate a list of RGBA colors

    y_cols_filtered = [col for col in y_cols if col not in ["worst_case"]]
    for i, y in enumerate(y_cols_filtered):
        # Assign a color based on the index (cycling if more than 10 functions)
        base_color = color_list[i % 10]
        
        # Generate colors for mean, min, and max
        mean_color = base_color  # Base color for the mean
        min_color = (base_color[0], base_color[1], base_color[2], 0.7)  # Adjust alpha for transparency
        max_color = (base_color[0], base_color[1], base_color[2], 1.0)  # Keep max fully opaque
        
        # Columns for aggregated data
        mean_col = f"{y}_mean"
        min_col = f"{y}_min"
        max_col = f"{y}_max"
        
        # Plot mean line
        sns.lineplot(data=aggregated_df, x=x_col, y=mean_col, label=f"{preprocess_label(y)} (Mean)", color=mean_color, errorbar=None)
        
        # Plot scatter points for min and max with their respective colors
        plt.scatter(aggregated_df[x_col], aggregated_df[min_col], color=min_color, label=f"{preprocess_label(y)} (Min)", alpha=0.7)
        plt.scatter(aggregated_df[x_col], aggregated_df[max_col], color=max_color, label=f"{preprocess_label(y)} (Max)", alpha=0.7)

    
    # Plot Opt column if available
    if opt_col:
        # Assign a unique color for the `opt_col` using an additional color index
        opt_color = color_list[len(y_cols) % 10]  # Next color in the sequence
    
        # Generate colors for mean, min, and max
        opt_mean_color = opt_color  # Base color for the mean
        opt_min_color = (opt_color[0], opt_color[1], opt_color[2], 0.7)  # Adjust alpha for transparency
        opt_max_color = (opt_color[0], opt_color[1], opt_color[2], 1.0)  # Keep max fully opaque
    
        # Columns for aggregated data
        opt_mean_col = f"{opt_col}_mean"
        opt_min_col = f"{opt_col}_min"
        opt_max_col = f"{opt_col}_max"
    
        # Plot mean line
        sns.lineplot(data=aggregated_df, x=x_col, y=opt_mean_col, label=f"{preprocess_label(opt_col)} (Mean)", linestyle="--", color=opt_mean_color, errorbar=None)
    
        # Plot scatter points for min and max with their respective colors
        plt.scatter(aggregated_df[x_col], aggregated_df[opt_min_col], color=opt_min_color, label=f"{opt_col} (Min)", alpha=0.7)
        plt.scatter(aggregated_df[x_col], aggregated_df[opt_max_col], color=opt_max_color, label=f"{opt_col} (Max)", alpha=0.7)

    # Check if "worst_case" is in y_cols
    if "worst_case" in y_cols:
        # Plot the worst-case mean line
        sns.lineplot(
            data=aggregated_df,
            x=x_col,
            y="worst_case_mean",
            label="Worst Case (Mean)",
            color="black",
            linestyle="-",
            linewidth=2.5,
            errorbar=None
        )
        
        # Plot scatter points for worst-case min and max
        plt.scatter(
            aggregated_df[x_col],
            aggregated_df["worst_case_min"],
            color="gray",
            label="Worst Case (Min)",
            alpha=0.7
        )
        plt.scatter(
            aggregated_df[x_col],
            aggregated_df["worst_case_max"],
            color="gray",
            label="Worst Case (Max)",
            alpha=1.0
        )
    if log_scale:
        plt.yscale("log")
    
    plt.legend()
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.savefig(output_file)
    plt.close()

In [12]:
def process_data(
    testset,
    dreyfus_Wagner=False,
    combined_df=None, # pandas data frame
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    worst_case_param = "NumberOfTerminals",
):
    # Prepare path and URL
    folder_path = os.path.join(results_folder, testset)

    if not dreyfus_Wagner:
        testset_url = base_url + testset
    
        
        # Fetch "Opt" values
        response = requests.get(testset_url)
        opt_values = {}
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            table_rows = soup.find_all("tr")[1:]
            for row in table_rows:
                cols = row.find_all("td")
                if len(cols) > 5:
                    instance_name = cols[0].text.strip()
                    opt_value = cols[-1].text.strip().replace("\xa0", "")
                    opt_values[instance_name] = float(opt_value) if opt_value.isdigit() else None
        else:
            print(f"Failed to fetch 'Opt' values for testset {testset}, status code: {response.status_code}")
            return pd.DataFrame()  # Return empty DataFrame if fetching fails

    # Process results files
    data_frames = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".results"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            if not dreyfus_Wagner:
                # instance_name = filename.split(".")[0]
                instance_name = filename.split(".")[0].lower()  # Convert instance_name to lowercase
                opt_values_lower = {key.lower(): value for key, value in opt_values.items()}  # Convert opt_values keys to lowercase
                if instance_name in opt_values_lower:
                    df["Opt"] = opt_values_lower[instance_name]
            
            
            time_columns = [col for col in df.columns if "Time" in col or "duration" in col]
            for col in time_columns:
                df[col] = df[col] / 1_000_000
            data_frames.append(df)


    # print(data_frames)
    if not data_frames:
        print(f"No .results files found in {folder_path}.")
        return pd.DataFrame()  # Return empty DataFrame if no results files are found

    combined_df = pd.concat(data_frames, ignore_index=True)
    
    # Calculate worst-case values
    if worst_case_param in combined_df.columns:
        combined_df["worst_case"] = 2 * (1 - 1 / combined_df[worst_case_param])
    else:
        print("Column '{worst_case_param}' not found in the data.")
        return pd.DataFrame()  # Return empty DataFrame if worst-case param column is missing


    opt = "Opt"
    if dreyfus_Wagner:
        opt = "DreyfusWagnerCost"

    min_time = min(combined_df["TakahashiMatsuyamaTime"].min(),combined_df["KouMarkowskyBermanTime"].min())
    combined_df["TMTimeToPerformance"] = ((combined_df["TakahashiMatsuyamaCost"]/combined_df[opt]) / (min_time / combined_df["TakahashiMatsuyamaTime"]))
    combined_df["KMBTimeToPerformance"] = ((combined_df["KouMarkowskyBermanCost"]/combined_df[opt]) / (min_time / combined_df["KouMarkowskyBermanTime"]))
    combined_df["DWTimeToPerformance"] = (combined_df["DreyfusWagnerCost"] / (combined_df["DreyfusWagnerTime"].min() / combined_df["DreyfusWagnerTime"]))


    # Check if at any instance we achieved optimal solution
    matching_rows_KMB = combined_df[
        (combined_df["KouMarkowskyBermanCost"] == combined_df[opt]) 
    ]
    matching_rows_TM = combined_df[
        (combined_df["TakahashiMatsuyamaCost"] == combined_df[opt])
    ]
    
    # Check if any rows matched
    if not matching_rows_KMB.empty:
        print("IMPORTANT: There are rows where cost equals the optimal value in KMB!")
    if not matching_rows_TM.empty:
        print("IMPORTANT: There are rows where cost equals the optimal value in TM!")  
              
    return combined_df

In [13]:
def process_and_plot(
    testset,
    dreyfus_Wagner=False,
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    output_folder="./overleaf-repo/images/plots",
    worst_case_param = "NumberOfTerminals",
    extra_testsets = None,
):
    combined_name = testset
    if not extra_testsets:
        output_folder = os.path.join(output_folder, testset)
    else:
        combined_name = "_".join([testset] + extra_testsets)  # Combine testset names
        output_folder = os.path.join(output_folder, combined_name)
        
    os.makedirs(output_folder, exist_ok=True)


    if dreyfus_Wagner:
        opt = "DreyfusWagnerCost"
    else:
        opt = "Opt"
        
    combined_df = process_data(
        testset=testset,
        dreyfus_Wagner=dreyfus_Wagner,
        results_folder=results_folder,
        base_url=base_url,
        worst_case_param=worst_case_param,
    )
    if combined_df is None or combined_df.empty:
        print(f"No valid data returned for testset: {combined_df}")
        return

    # Process additional testsets if provided
    if extra_testsets:
        for additional_testset in extra_testsets:
            additional_df = process_data(
                testset=additional_testset,
                dreyfus_Wagner=dreyfus_Wagner,
                combined_df=combined_df,
                results_folder=results_folder,
                base_url=base_url,
                worst_case_param=worst_case_param,
            )
            # Check if the returned DataFrame is not None or empty
            if additional_df is not None and not additional_df.empty:
                if combined_df is None or combined_df.empty:
                    combined_df = additional_df  # Initialize combined_df if it was empty
                else:
                    combined_df = pd.concat([combined_df, additional_df], ignore_index=True)
            else:
                print(f"No valid data returned for testset: {additional_testset}")
                return
    
    # Extract the subdirectory (e.g., 'B' from './overleaf-repo/images/plots/B')
    subdirectory = get_subdirectory_from_path(output_folder)

    # Set the new output folder path for tables (e.g., './overleaf-repo/tables/B')
    tables_output_folder = os.path.join("./overleaf-repo/tables", subdirectory)

    set_of_x_axes = ["NumberOfTerminals", "NumberOfNodes", "NumberOfEdges"]

    for variable in set_of_x_axes:
        # Generate plots
        # Plot Costs by variable
        plot_data(
            combined_df,        
            x_col=variable,
            y_cols=["TakahashiMatsuyamaCost", "KouMarkowskyBermanCost", "worst_case"],
            title=f"Costs vs {preprocess_label(variable)} ({combined_name})",
            x_label=preprocess_label(variable),
            y_label="Cost",
            output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_cost.png"),
            opt_col=opt
        )
        # Generate tables with costs
        per_x_stats, overall_stats_df = create_summary_tables(
            combined_df,        
            x_col=variable,
            y_cols=["TakahashiMatsuyamaCost", "KouMarkowskyBermanCost"],
            opt_col=opt,
            output_folder=tables_output_folder
        )
        # Generate tables with approximation ratio
        per_x_ratios, overall_ratios = create_approximation_ratio_table(
            df=combined_df,        
            x_col=variable,
            y_cols=["TakahashiMatsuyamaCost", "KouMarkowskyBermanCost", "worst_case"],
            opt_col=opt,
            output_folder=tables_output_folder
        )
        # Plot Times by variable
        plot_data(
            combined_df,
            x_col=variable,
            y_cols=["TakahashiMatsuyamaTime", "KouMarkowskyBermanTime"],
            title=f"Times vs {preprocess_label(variable)} ({combined_name})",
            x_label=preprocess_label(variable),
            y_label="Time (seconds)",
            output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_time.png")
        )
        # Time to performence
        plot_data(
            combined_df,
            x_col=variable,
            y_cols=["TMTimeToPerformance", "KMBTimeToPerformance"],
            title=f"Performance Cost vs {preprocess_label(variable)} ({combined_name})",
            x_label=preprocess_label(variable),
            y_label="Performance Cost",
            output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_CostToTime.png")
        )

    if dreyfus_Wagner:
        for variable in set_of_x_axes:
            # Generate plots
            # Plot Times by variable
            plot_data(
                combined_df,
                x_col=variable,
                y_cols=["TakahashiMatsuyamaTime", "KouMarkowskyBermanTime","DreyfusWagnerTime"],
                title=f"Times vs {preprocess_label(variable)} ({combined_name})",
                x_label=preprocess_label(variable),
                y_label="Time (seconds)",
                output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_time_with_DW.png")
            )
            # Time to performence
            plot_data(
                combined_df,
                x_col=variable,
                y_cols=["TMTimeToPerformance", "KMBTimeToPerformance","DWTimeToPerformance"],
                title=f"Performance Cost vs {preprocess_label(variable)} ({combined_name})",
                x_label=preprocess_label(variable),
                y_label="Performance Cost",
                output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_CostToTime_with_DW.png")
            )

    # Additional plots for TMduration* and KMBduration*
    tm_duration_columns = [col for col in combined_df.columns if col.startswith("TMduration")]
    kmb_duration_columns = [col for col in combined_df.columns if col.startswith("KMBduration")]

    for variable in set_of_x_axes:
        # Inner Times
        plot_data(
            combined_df,        
            x_col=variable,
            y_cols=tm_duration_columns,
            title=f"TM Durations vs {preprocess_label(variable)} ({combined_name})",
            x_label=preprocess_label(variable),
            y_label="Time (seconds)",
            output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_tm_durations.png")
        )
    
        plot_data(
            combined_df,        
            x_col=variable,
            y_cols=kmb_duration_columns,
            title=f"KMB Durations vs {preprocess_label(variable)} ({combined_name})",
            x_label=preprocess_label(variable),
            y_label="Time (seconds)",
            output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_kmb_durations.png"),
        )
        
        plot_data(
            combined_df,        
            x_col=variable,
            y_cols=kmb_duration_columns,
            title=f"KMB Durations vs {preprocess_label(variable)} ({combined_name})",
            x_label=preprocess_label(variable),
            y_label="Time (seconds)",
            output_file=os.path.join(output_folder, f"{split_after_second_space(variable)}_kmb_durations_log_scale.png"),
            log_scale=True
        )

    print(f"All plots for testset {testset} have been generated in {output_folder}.")

In [14]:
process_and_plot("B")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/B/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/B/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/B/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/B/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/B/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/B/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/B/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/B/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/B/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/B/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/B/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/

In [144]:
process_and_plot("C")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/C/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/C/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/C/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/C/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/C/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/C/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/C/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/C/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/C/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/C/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/C/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/

In [145]:
process_and_plot("D")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
Summary tables saved: ./overleaf-repo/tables/D/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/D/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/D/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/D/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/D/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/D/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/D/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/D/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/D/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/D/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/D/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/D/approx_ratios_overall_NumberOfEdges.tex
All plots for testse

In [146]:
process_and_plot("E")

Summary tables saved: ./overleaf-repo/tables/E/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/E/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/E/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/E/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/E/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/E/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/E/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/E/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/E/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/E/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/E/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/E/approx_ratios_overall_NumberOfEdges.tex
All plots for testset E have been generated in ./overleaf-repo/images/plots/E.


In [147]:
process_and_plot("I080")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/I080/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/I080/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I080/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/I080/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/I080/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/I080/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I080/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/I080/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/I080/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/I080/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I080/approx_ratios_per_x_Nu

In [148]:
process_and_plot("I160")

Summary tables saved: ./overleaf-repo/tables/I160/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/I160/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I160/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/I160/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/I160/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/I160/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I160/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/I160/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/I160/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/I160/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I160/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/I160/approx_ratios_overall_NumberOfEdges.tex
All plots for testset I160 have been generated in ./ov

In [149]:
process_and_plot("I320")

Summary tables saved: ./overleaf-repo/tables/I320/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/I320/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I320/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/I320/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/I320/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/I320/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I320/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/I320/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/I320/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/I320/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I320/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/I320/approx_ratios_overall_NumberOfEdges.tex
All plots for testset I320 have been generated in ./ov

In [150]:
process_and_plot("I640")

Summary tables saved: ./overleaf-repo/tables/I640/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/I640/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I640/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/I640/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/I640/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/I640/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I640/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/I640/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/I640/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/I640/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I640/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/I640/approx_ratios_overall_NumberOfEdges.tex
All plots for testset I640 have been generated in ./ov

In [166]:
process_and_plot("P4E")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
Summary tables saved: ./overleaf-repo/tables/P4E/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/P4E/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4E/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/P4E/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/P4E/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/P4E/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4E/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/P4E/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/P4E/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/P4E/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4E/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/P4E/approx_ratios_overall_NumberOfEdges.

/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")
/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


All plots for testset P4E have been generated in ./overleaf-repo/images/plots/P4E.


/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


In [167]:
process_and_plot("P4Z")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/P4Z/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/P4Z/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4Z/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/P4Z/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/P4Z/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/P4Z/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4Z/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/P4Z/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/P4Z/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/P4Z/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4Z/approx_ratios_per_x_NumberOfEdges

/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")
/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


All plots for testset P4Z have been generated in ./overleaf-repo/images/plots/P4Z.


/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


In [168]:
process_and_plot("P6E")

Summary tables saved: ./overleaf-repo/tables/P6E/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/P6E/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P6E/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/P6E/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/P6E/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/P6E/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P6E/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/P6E/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/P6E/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/P6E/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P6E/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/P6E/approx_ratios_overall_NumberOfEdges.tex


/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")
/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


All plots for testset P6E have been generated in ./overleaf-repo/images/plots/P6E.


/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


In [169]:
process_and_plot("P6Z")

Failed to fetch 'Opt' values for testset P6Z, status code: 500
No valid data returned for testset: Empty DataFrame
Columns: []
Index: []


In [170]:
process_and_plot(
    testset = "B",
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    output_folder="./overleaf-repo/images/plots",
    worst_case_param = "NumberOfTerminals",
    extra_testsets = ["C","D","E","I080","I160","I320","I640"]
)

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/B_C_D_E_I080_I160_I320_I640/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/B_C_D_E_I080_I160_I320_I640/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/B_C_D_E_I080_I160_I320_I640/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/B_C_D_E_I080_I160_I320_I640/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/B_C_D_E_I080_I160_I320_I640/per_x_stats_NumberOfNo

In [156]:
process_and_plot(
    testset = "B",
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    output_folder="./overleaf-repo/images/plots",
    worst_case_param = "NumberOfTerminals",
    extra_testsets = ["C","D","E",]
)

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
Summary tables saved: ./overleaf-repo/tables/B_C_D_E/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/B_C_D_E/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/B_C_D_E/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/B_C_D_E/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/B_C_D_E/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/B_C_D_E/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/B_C_D_E/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/B_C_D_E/approx_ratios_overall_NumberOfNo

In [157]:
process_and_plot(
    testset = "I080",
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    output_folder="./overleaf-repo/images/plots",
    worst_case_param = "NumberOfTerminals",
    extra_testsets = ["I160","I320","I640"]
)

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/I080_I160_I320_I640/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/I080_I160_I320_I640/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I080_I160_I320_I640/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/I080_I160_I320_I640/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/I080_I160_I320_I640/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/I080_I160_I320_I640/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/I080_I160_I320_I640/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/I080_I160_I320_I640/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/I080_I160_I320_I640/per_x_stats_NumberOfEdges.tex, ./overlea

In [158]:
process_and_plot(
    testset = "P4E",
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    output_folder="./overleaf-repo/images/plots",
    worst_case_param = "NumberOfTerminals",
    extra_testsets = ["P4Z"]
)

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/P4E_P4Z/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/P4E_P4Z/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4E_P4Z/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/P4E_P4Z/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/P4E_P4Z/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/P4E_P4Z/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/P4E_P4Z/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/P4E_P4Z/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/P4E_P4Z/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/P4E_P4Z/overall_stats_Nu

In [159]:
process_and_plot("PUC")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
Summary tables saved: ./overleaf-repo/tables/PUC/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/PUC/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/PUC/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/PUC/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/PUC/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/PUC/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/PUC/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/PUC/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/PUC/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/PUC/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/PUC/approx_ratios_per_x_NumberOfEdges.tex, ./overleaf-repo/tables/PUC/approx_ratios_overall_NumberOfEdges.

/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")
/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


All plots for testset PUC have been generated in ./overleaf-repo/images/plots/PUC.


/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


In [160]:
process_and_plot("LIN")

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/LIN/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/LIN/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/LIN/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/LIN/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/LIN/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/LIN/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/LIN/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/LIN/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/LIN/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/LIN/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/LIN/approx_ratios_per_x_NumberOfEdges

/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")
/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


All plots for testset LIN have been generated in ./overleaf-repo/images/plots/LIN.


/tmp/ipykernel_386/1742335589.py:104: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.yscale("log")


In [161]:
process_and_plot(
    testset = "B",
    results_folder="./results/GraphInstances",
    base_url="https://steinlib.zib.de/showset.php?",
    output_folder="./overleaf-repo/images/plots",
    worst_case_param = "NumberOfTerminals",
    extra_testsets = ["C","D","E","I080","I160","I320","I640","PUC","LIN","P4E","P4Z","P6E"]
)

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/B_C_D_E_I080_I160_I320_I640_PUC_LIN_P4E_P4Z_P6E/pe

In [15]:
process_and_plot("OurOwn/d05", dreyfus_Wagner=True)

IMPORTANT: There are rows where cost equals the optimal value in KMB!
IMPORTANT: There are rows where cost equals the optimal value in TM!
Summary tables saved: ./overleaf-repo/tables/d05/per_x_stats_NumberOfTerminals.tex, ./overleaf-repo/tables/d05/overall_stats_NumberOfTerminals.tex
Approximation ratio tables saved: ./overleaf-repo/tables/d05/approx_ratios_per_x_NumberOfTerminals.tex, ./overleaf-repo/tables/d05/approx_ratios_overall_NumberOfTerminals.tex
Summary tables saved: ./overleaf-repo/tables/d05/per_x_stats_NumberOfNodes.tex, ./overleaf-repo/tables/d05/overall_stats_NumberOfNodes.tex
Approximation ratio tables saved: ./overleaf-repo/tables/d05/approx_ratios_per_x_NumberOfNodes.tex, ./overleaf-repo/tables/d05/approx_ratios_overall_NumberOfNodes.tex
Summary tables saved: ./overleaf-repo/tables/d05/per_x_stats_NumberOfEdges.tex, ./overleaf-repo/tables/d05/overall_stats_NumberOfEdges.tex
Approximation ratio tables saved: ./overleaf-repo/tables/d05/approx_ratios_per_x_NumberOfEdges

In [ ]:
process_and_plot("OurOwn/d08", dreyfus_Wagner=True)

In [ ]:
# process_and_plot(
#     testset = "B",
#     results_folder="./results/GraphInstances",
#     base_url="https://steinlib.zib.de/showset.php?",
#     output_folder="./overleaf-repo/images/plots",
#     worst_case_param = "NumberOfTerminals",
#     extra_testsets = ["C","D","E","I080","I160","I320","I640","PUC","LIN","P4E","P4Z","P6E","d05","d08"]
# )